In [1]:
# PYTHON IMPORTS
import os
import copy
from tqdm.notebook import trange, tqdm

# IMAGE IMPORTS 
from PIL import Image
import cv2

# DATA IMPORTS 
import random
import h5py
import numpy as np

# PLOTTING
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# NEURAL NETWORK
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from torchvision.transforms import ToPILImage, GaussianBlur
from torchvision.transforms import Compose, RandomCrop, ToTensor, Normalize
import torch.optim.lr_scheduler as lr_scheduler
import torchvision.models as models
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# MY OWN CLASSES
from TileLocator import *
Image.MAX_IMAGE_PIXELS = 933120000

In [2]:
def resize_images(directory_a, directory_b, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for filename in os.listdir(directory_a):
        image_a_path = os.path.join(directory_a, filename)
        image_b_path = os.path.join(directory_b, filename)
        output_path = os.path.join(output_directory, filename)

        if os.path.isfile(image_a_path) and os.path.isfile(image_b_path):
            image_a = Image.open(image_a_path)
            image_b = Image.open(image_b_path)

            # Resize image_a to the dimensions of image_b
            image_a_resized = image_a.resize(image_b.size, Image.ANTIALIAS)

            # Save the resized image to the output directory
            image_a_resized.save(output_path)
        else:
            print(f"Image pair {filename} not found in both directories.")

# Example usage:
directory_b = r"C:\Users\fhacesga\OneDrive - University Of Houston\AAA_RECTDNN\data\TileLocator\out\tiles"
directory_a = r"C:\Users\fhacesga\OneDrive - University Of Houston\AAA_RECTDNN\data\TileLocator\roads_big"
output_directory = r"C:\Users\fhacesga\OneDrive - University Of Houston\AAA_RECTDNN\data\TileLocator\out\roads"

if False:
    resize_images(directory_a, directory_b, output_directory)

In [3]:
base_dir = r"C:\Users\fhacesga\OneDrive - University Of Houston\AAA_RECTDNN\data"

input_folder = f"{base_dir}/TileLocator/in"
val_folder = f"{base_dir}/TileLocator/in"
target_folder = f"{base_dir}/TileLocator/out"
batch_size = 2

transform = transforms.Compose([
    transforms.RandomRotation(degrees=180),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
])

tensor = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = SegmentationDataset_Multiclass(input_folder, target_folder, transform=transform, crop=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# val_dataset = SegmentationDataset_Multiclass(val_folder, target_folder, transform=transform, crop=True)
# val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=0)

loaders = {'train' : train_loader}

In [4]:
def train(model, dataloaders, num_epochs=50, 
          output_dir=f'{base_dir}/TileLocator/intermediate_outputs', 
          learning_rate=5e-4):
    device = torch.device("cuda:0")
    weights = torch.tensor([1, 120, 120]).float().to(device)
    criterion = nn.CrossEntropyLoss(weight=weights, 
        reduction="mean")
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    learning_rate_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.97)
    
    
    
    model = model.to(device)
    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        
        for phase in ['train']: 
            if phase == 'train':
                model.train()
                repeats = range(4)
            else:
                model.eval()
                repeats = range(2)

            running_loss = 0.0
            
            outputs_folder = os.path.join(output_dir, phase)
            if not os.path.exists(outputs_folder):
                os.makedirs(outputs_folder)
                
            curr_loss = 0
            
            # Iterate over data.
            for rep_id in tqdm(repeats):
                if rep_id % 2 == 0 and rep_id != 0:
                    print(f"{curr_loss:.4e} {curr_loss/8:.4e}")
                    curr_loss = 0
                for inputs, labels, filenames in dataloaders[phase]:
                    
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    
                    print(inputs.dtype)
                    print(labels.dtype)
                    
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    
                    print(outputs.dtype)
                    
                    loss = criterion(outputs, labels) 
                    curr_loss += loss
                    if rep_id % 2 == 0 and phase is 'train' and rep_id != 0:
                        prob_img_or = outputs.detach().cpu()
                        minputs = inputs.detach().cpu().numpy()
                        
                        if prob_img_or.ndim == 3:
                            prob_img_or = prob_img_or.unsqueeze(0)
                            
                        prob_img_or = prob_img_or.numpy()
                        
                        for i in range(len(outputs)):
                            filename = filenames[i]
                            for ii in range(prob_img_or.shape[1]):
                                prob_img = prob_img_or[i, ii, :, :]
                                prob_img = (prob_img * 255).astype(np.uint8)
                                prob_img = Image.fromarray(np.squeeze(prob_img))
                                prob_img.save(os.path.join(outputs_folder, f"{rep_id}_{ii}_{filename}"))
                            myinp = Image.fromarray(np.uint8(minputs[i, 0, :, :] * 255))
                            myinp.save(os.path.join(outputs_folder, f"{rep_id}_{filename[:-3]}_inp.png"))
                    if phase is 'train':
                        loss.backward()
                        optimizer.step()
                    running_loss += loss.item()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f}'.format(phase, epoch_loss))

            if phase == 'val' and epoch_loss < best_acc:
                best_model_wts = copy.deepcopy(model.state_dict())
        # Update the learning rate scheduler after each epoch
        learning_rate_scheduler.step()
        print(f"Learning Rate: {optimizer.param_groups[0]['lr']}")
        # Save the model and optimizer states
        if epoch % 10 == 0:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
            }, f'{base_dir}/TileLocator/checkpoint_072523.pth')
            
            torch.save(model, f"{base_dir}/TileLocator/072523.pth")
        
    return model

In [5]:
model = RectangleClass(num_classes=3)
model = train(model, loaders, num_epochs=500, learning_rate=1e-3)

C:\Users\fhacesga\Anaconda3\envs\deeplearning\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
C:\Users\fhacesga\Anaconda3\envs\deeplearning\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/500
----------


  0%|          | 0/4 [00:00<?, ?it/s]

torch.float32
torch.int64


C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\TileLocator.py:93: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)


torch.float32
torch.float32
torch.int64
torch.float32
torch.float32
torch.int64
torch.float32
torch.float32
torch.int64
torch.float32


KeyboardInterrupt: 